In [172]:
import pymolpro
import pandas as pd

In [173]:
backend = 'local' 
project_name = 'INV24_methodtest'
parallel = 8

In [174]:
methods = {"HF": "hf",
"MP2": "mp2",
"MP3": "mp3",
"MP4": "mp4",
"PAPT2":"PAPT;MP2",
"PAPT3":"PAPT;MP3",
"PAPT4":"PAPT;MP4",
"CCSD":"CCSD",
"CCSD(T)":"CCSD(T)" }
bases = ['cc-pVDZ','cc-pVTZ']

In [175]:
db = pymolpro.database.load("GMTKN55_INV24")

In [176]:
small=db.subset(open_shell=False,max_electrons=30)
print(small)

GMTKN55 INV24 (closed shell only) (maximum number of electrons 30):

Molecules:
H2O: {'geometry': '3\n  \n   O   -0.02085472758024      0.00000000000000     -0.01495598121153\n   H   0.01876805202920      0.00000000000000      0.94578209125484\n   H   0.89775667555104      0.00000000000000     -0.29748911004330\n', 'description': 'H2O'}
H2S: {'geometry': '3\n  \n   S   -0.20556127464025      0.00000000000000     -0.14507162979824\n   H   -0.01007104299822      0.00000000000000      1.18200447183515\n   H   1.11130231763847      0.00000000000000     -0.40359584203690\n', 'description': 'H2S'}
Ether_TS: {'geometry': '9\n  \n   O   -0.00020946572717      0.00033803651382      0.01009578182224\n   C   1.37497499316945      0.00000551844024      0.01033708407522\n   C   -1.37521529311648      0.00068147573508      0.00978328762624\n   H   1.76662285429289      0.40690776018797     -0.93108108766811\n   H   1.76646977000414      0.61379439668452      0.82991943080001\n   H   1.77133317507623

In [ ]:
results = {}
for method in methods:
    results[method] = {}
    for basis in bases:
        results[method][basis] = pymolpro.database.run(small, methods[method], basis, location=project_name,
                                                       backend=backend,
                                                       preamble="core,small", parallel=parallel)
        if results[method][basis].failed: print(method, basis, 'failed', results[method][basis].project_directory)
        
print(results)       

In [ ]:
analyse=pymolpro.database.analyse([result['cc-pVTZ'] for result in results.values()],small)
analyse['reaction statistics']

In [ ]:
extrapolate=pymolpro.database.basis_extrapolate(results[method].values(), results["HF"].values())
print(extrapolate)

In [ ]:
print(extrapolate[0])

In [ ]:
import matplotlib.pyplot as plt

methods_pruned = [method for method in methods if (method != 'HF') and (method != 'CCSD(T)')]
bases_pruned = ['cc-pVTZ', 'cc-pVDZ']
fig, panes = plt.subplots(nrows=1, ncols=len(bases_pruned), sharey=True, figsize=(18, 6))

for pane in range(len(bases_pruned)):
    data = []
    for method in methods_pruned:
        data.append(
            pymolpro.database.analyse(results[method][bases_pruned[pane]],
                                      results['CCSD(T)']['cc-pVTZ'],'kJ/mol')['reaction energy deviations'].to_numpy()[:, 0]
        )
    panes[pane].violinplot(data, showmeans=True, showextrema=True, vert=True, bw_method='silverman')
    panes[pane].set_xticks(range(1, len(methods_pruned) + 1), labels=methods_pruned, rotation=-90)
    panes[pane].set_title(bases_pruned[pane])
panes[0].set_ylabel('Error / kJ/mol')
plt.savefig(project_name + ".violin.pdf")

df = pd.DataFrame(data)
df.to_excel("PAPT.xlsx")


In [161]:
print(extrapolate[0])

Database:

Molecules:
H2O: {'geometry': '3\n  \n   O   -0.02085472758024      0.00000000000000     -0.01495598121153\n   H   0.01876805202920      0.00000000000000      0.94578209125484\n   H   0.89775667555104      0.00000000000000     -0.29748911004330\n', 'description': 'H2O'}, energy = -76.36634776415646
H2S: {'geometry': '3\n  \n   S   -0.20556127464025      0.00000000000000     -0.14507162979824\n   H   -0.01007104299822      0.00000000000000      1.18200447183515\n   H   1.11130231763847      0.00000000000000     -0.40359584203690\n', 'description': 'H2S'}, energy = -398.99337593548154
Ether_TS: {'geometry': '9\n  \n   O   -0.00020946572717      0.00033803651382      0.01009578182224\n   C   1.37497499316945      0.00000551844024      0.01033708407522\n   C   -1.37521529311648      0.00068147573508      0.00978328762624\n   H   1.76662285429289      0.40690776018797     -0.93108108766811\n   H   1.76646977000414      0.61379439668452      0.82991943080001\n   H   1.7713331750762

In [160]:
len(extrapolate)

1